In [1]:
import torch
from shiftgcn import get_shift_indexes, get_shift_indexes_with_test_case

# Writing

In [2]:
device = 'cpu'

N = 32
V = 25
A = torch.eye(V, device = device).unsqueeze(0).repeat(N, 1, 1) 
# A = torch.ones((N, V, V), device = device)
C = A.softmax(dim = -1).to(device).view(-1, V)

KERNEL = 256

# shift_indexes  = get_shift_indexes(C, 256, device)

In [3]:
counts = C * KERNEL
rounded_counts = torch.floor(counts).long()
remaining_count = KERNEL - torch.sum(rounded_counts, dim = -1)
fractional_parts = counts - rounded_counts
fractional_parts.shape

torch.Size([800, 25])

In [4]:
remaining_count.shape

torch.Size([800])

In [5]:
sorted_indices = torch.argsort(fractional_parts, dim = -1, descending=True)
sorted_indices

tensor([[12, 24, 23,  ...,  2,  1,  0],
        [12, 24, 23,  ...,  3,  2,  1],
        [12, 24, 23,  ...,  3,  1,  2],
        ...,
        [12, 24, 23,  ...,  2,  1, 22],
        [12, 24, 22,  ...,  2,  1, 23],
        [ 1, 23, 22,  ...,  2,  0, 24]])

In [6]:
fake_mask = torch.arange(V).reshape(1, V).repeat(N * V, 1) < remaining_count.unsqueeze(1)
fake_mask.shape

torch.Size([800, 25])

In [7]:
fake_mask.shape

torch.Size([800, 25])

In [8]:
real_mask = torch.gather(fake_mask, 1, sorted_indices.argsort())

In [9]:
final_counts = torch.zeros((N * V, V), dtype=torch.long).masked_scatter(real_mask, torch.ones((N * V, V), dtype=torch.long)) + rounded_counts

In [10]:
final_counts[0]

tensor([26,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10])

In [11]:
vertexes_tensor = torch.arange(0, V, device=device, requires_grad= False).repeat(N * V)

shift_indexes = torch.repeat_interleave(vertexes_tensor, final_counts.view(-1)).view(N, V, -1)

position_indexes = torch.arange(KERNEL, dtype=torch.long, device=device, requires_grad=False).reshape((1, 1, -1)).repeat(N, V, 1)

# Test case

In [32]:
device = 'cpu'

N = 32
V = 25
A = torch.eye(V, device = device).unsqueeze(0).repeat(N, 1, 1)
# A = torch.ones((N, V, V), device = device)
C = A.softmax(dim = -1).to(device)

KERNEL = 6

In [33]:

shift_indexes = get_shift_indexes(C, KERNEL, 'cpu')

In [34]:
shift_indexes[0]

tensor([[  0,  73, 128, 135, 142, 149],
        [  0,   7, 128, 135, 142, 149],
        [ 12,  73, 128, 135, 142, 149],
        [ 18,  73, 128, 135, 142, 149],
        [ 24,  73, 128, 135, 142, 149],
        [ 30,  73, 128, 135, 142, 149],
        [ 36,  73, 128, 135, 142, 149],
        [ 42,  73, 128, 135, 142, 149],
        [ 48,  73, 128, 135, 142, 149],
        [ 54,  73, 128, 135, 142, 149],
        [ 60,  73, 128, 135, 142, 149],
        [ 66,  73, 128, 135, 142, 149],
        [  0,  73, 128, 135, 142, 149],
        [ 72,  79, 128, 135, 142, 149],
        [ 72,  85, 128, 135, 142, 149],
        [ 72,  91, 128, 135, 142, 149],
        [ 72,  97, 128, 135, 142, 149],
        [ 72, 103, 128, 135, 142, 149],
        [ 72, 109, 128, 135, 142, 149],
        [ 72, 115, 128, 135, 142, 149],
        [ 72, 121, 128, 135, 142, 149],
        [ 72, 121, 128, 135, 142, 149],
        [ 72, 121, 128, 135, 142, 149],
        [ 72, 121, 128, 135, 142, 149],
        [ 72, 121, 128, 135, 142, 149]])